In [1]:
import os
import json
import minsearch # !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
from openai import OpenAI
client = OpenAI()

# Load Referenced Document

In [2]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)
index.fit(documents)

# Build RAG LLM

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
        you are a course teaching assistant. 
        Answer the question based on the context. 
        use only the facts from the context when answering the question.
        if the context does not contain the answer, output none
        
        question: {question}
        context: {context}
        """

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context.strip())
    return prompt

In [7]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user","content":prompt}]
    )
    return response.choices[0].message.content

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [12]:
rag('the course already started. can i still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."